# 1.Data Import and Combination

In [1]:
import pandas as pd
import gensim as gs
import nltk
import re
from nltk.corpus import stopwords
import sklearn as sk
import numpy as np
import pickle
from tqdm import tqdm

## 1.1.import dataset

In [2]:
train_path="DataSet\\train.txt"
test_path="DataSet\\test.txt"
train_df = pd.read_csv(train_path,sep="\t",index_col=0)
test_df=pd.read_csv(test_path,sep="\t",index_col=0)

In [3]:
train_df

,turn1,turn2,turn3,label
id,,,,
0,Don't worry I'm girl,hmm how do I know if you are,What's ur name?,others
1,When did I?,saw many times i think -_-,No. I never saw you,angry
2,By,by Google Chrome,Where you live,others
3,U r ridiculous,I might be ridiculous but I am telling the truth.,U little disgusting whore,angry
4,Just for time pass,wt do u do 4 a living then,Maybe,others
...,...,...,...,...
30155,I don't work,I could take your shift,I am a student,others
30156,I'm not getting you 😭😭😭,Why are you crying??,Because you are not making any sense,sad
30157,Haha,"no, seriously. What is up with that o-o",Had your breakfast?,others


## 1.2.unify the variables for change

In [4]:
# now when I copy the code for another dataset, I can easily change the special name of the dataset for adjusting the process and storage
process_df = train_df
version_change = "word2vec"
path_change = "train"

In [5]:
emo_path="DataSet\\emo_collection_{b}.csv".format(b=version_change)
emo_df=pd.read_csv(emo_path,sep="|")

In [6]:
emo_df

,emoji/emoticon,key words
0,😀,face grin grinning face
1,😃,face grinning face big eyes mouth open smile
2,😄,eye face grinning face smiling eyes mouth ope...
3,😁,beaming face smiling eyes eye face grin smile
4,😆,face grinning squinting face laugh mouth sati...
...,...,...
2279,( ´Д｀)ﾉ(´･ω･`) ﾅﾃﾞﾅﾃﾞ,patting made made
2280,(*ﾟﾉOﾟ)<ｵｵｵｵｫｫｫｫｫｫｫｰｰｰｰｰｲ!,calling ooooh
2281,( ﾟ∀ﾟ)ｱﾊﾊ八八ﾉヽﾉヽﾉヽﾉ ＼ / ＼/ ＼,evil laugh literally ahahahaha
2282,（・∀・ ）ヾ(- -；)コラコラ,blaming


## 1.3.get label and turn into category

In [7]:
# set the label into categorical number:
# others:0; happy:1,angry:3;sad:2;

class Label_format:
    def __init__(self,df):
        self.df=df
        
    def get_y(self):
        y = self.df.iloc[:,-1].values
        return y
    
    def label_category(self,y):
        change_list = list()
        emotion = {"others":0,"happy":1,"sad":2,"angry":3}
        for each in y:
            change_list.append(emotion[each])
        change_array=np.array(change_list)
        return change_array
    
    def save_file(self,data,name):
        pickle_file = open('DataSet\\'+name,mode='wb')
        pickle.dump(data,pickle_file)
        pickle_file.close()

In [8]:
#set y label, and we make X later on.

lf = Label_format(process_df)
y = lf.get_y()
y = lf.label_category(y)
save_path = "{a}_y.pkl".format(a=path_change)
lf.save_file(y,save_path)

In [9]:
y

array([0, 3, 0, ..., 0, 0, 0])

## 1.4. class for combination

In [10]:
# because we use pre-trained embeddings, we can't directly use stemming, remove stopwords or remove punctuations.
# we need to turn the data as much as possible close to the pre-trained embedding.
def basic_clean_text(input_text):
    'clean text function'
    #clean emoji, punctions
    input_text = re.sub(r'[^a-zA-Z]',repl=" ",string=input_text)
    #lower the words
    input_text =input_text.lower()
    #stemming the words
    list_text=input_text.split()
    ps=nltk.stem.porter.PorterStemmer()
    list_text_out=list()
    for word in list_text:
        if word in set(stopwords.words('english')):
            continue
        list_text_out.append(ps.stem(word))
    input_text=" ".join(list_text_out) 
    return input_text

def combine_method(df):
    turn1_corpus= list(df["turn1"])
    turn2_corpus= list(df["turn2"])
    turn3_corpus= list(df["turn3"])
    
    # use text_content as dict for storing the data of 
    text_content={"turn1":turn1_corpus,"turn2":turn2_corpus,"turn3":turn3_corpus}
    
    # because we need all the sentences including sentences in turn1, turn2, turn3 
    # to predict the emtion situation, therefore we need to combine them together
    length = len(text_content["turn1"])
    text_all_content = list()
    for i in tqdm(range(length)):
        text_row_all = text_content["turn1"][i] +" " + text_content["turn2"][i] +" "+ text_content["turn3"][i]
        text_all_content.append(text_row_all)
    text_all_content = {"all":text_all_content}
    
    return text_content,text_all_content

In [11]:
text_3,text_all = combine_method(process_df)

100%|███████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 1163134.75it/s]


In [12]:
text_3

{'turn1': ["Don't worry  I'm girl",
  'When did I?',
  'By',
  'U r ridiculous',
  'Just for time pass',
  "I'm a dog person",
  'So whatsup',
  'Ok',
  'Really?',
  'Bay',
  'I hate my boyfriend',
  'I will do night.',
  'Sure go ahead',
  'Bad',
  'Ok get it......',
  'Money money and lots of money😍😍',
  'My gf left ne',
  'get lost',
  'You are lying and i know that',
  'Ur creator is very bad',
  'ehat is hehe',
  'Do you dance?',
  'I hate it too',
  'Not always',
  'Bcoz u dont know wat is to miss someone',
  'Yeah',
  'I want to tell you something',
  'Yes I will send the email and you write all yours filling still I write all matters but your are clever and not any matter for yourself please send andwrite',
  'You are very funny',
  'wth',
  'so rude',
  'ok i will',
  'Ntng.....',
  'You are fool',
  'U dare me?',
  'Abt me',
  'crazy',
  'Goa in India',
  'That was mean',
  'yes :)',
  "I don't no",
  'Send me your nekde pic',
  "OK friend's",
  'She is ignoring me',
  'Happy

In [13]:
text_all

{'all': ["Don't worry  I'm girl hmm how do I know if you are What's ur name?",
  'When did I? saw many times i think -_- No. I never saw you',
  'By by Google Chrome Where you live',
  'U r ridiculous I might be ridiculous but I am telling the truth. U little disgusting whore',
  'Just for time pass wt do u do 4 a living then Maybe',
  "I'm a dog person youre so rude Whaaaat why",
  'So whatsup Nothing much. Sitting sipping and watching TV. How abt u? What are you watching on tv?',
  'Ok ok im back!! So, how are u',
  'Really? really really really really really Y saying so many times...i can hear you',
  'Bay in the bay 😘 love you',
  'I hate my boyfriend you got a boyfriend? Yes',
  'I will do night. Alright. Keep me in loop. Not giving WhatsApp no.',
  'Sure go ahead Many thanks once again! Love you too',
  "Bad Bad bad! That's the bad kind of bad. I have no gf",
  'Ok get it...... I made it an option Ok',
  "Money money and lots of money😍😍 I need to get it tailored but I'm in love w

## 1.5. Main Function

In [14]:
def fit_transform(input_dict,method):
    output_dict=dict()
    for key in input_dict.keys():
        processed_texts = list()
        for text in tqdm(input_dict[key]):
            processed_single_text = eval(method)(text)
            processed_texts.append(processed_single_text)
        
        output_dict[key]=processed_texts
    
    return output_dict

# 2. Preprocessing for cleaning the data
## 2.1. Extract emoji, emoticons based on emo_df, and build 2 versions of data
- one version for saving the extracted emoji, emoticons of each sentence
- one version for text without the emoji, emoticons based on emo_df

In [15]:
emo_values = list(emo_df["emoji/emoticon"])
meaning_values = list(emo_df["key words"])

def count_emo(sentence,emo):
    'function for counting how many same emoji or emoticon in one sentence'
    sentence = sentence.replace(emo," count_no ")
    words_list = sentence.split()
    count_no=0
    for word in words_list:
        if "count_no" in word:
            count_no +=1
    
    return count_no
            

def extract_emo(text_dict,emo_values,meaning_values):
    calculate_emo=dict()
    del_emo_dict=dict()
    for key in text_dict.keys():
        length = len(text_dict[key])
        del_emo_list = text_dict[key].copy()
        
        # set list_all_emo as the storage for storing all emoji and emoticons for all sentences
        list_all_emo = list()

        len_emo = len(emo_values)
        for i in tqdm(range(length)):
            
            # set list_emo as the storage for storing the emo for each sentence
            list_emo = list()
            for e in range(len_emo):
                if emo_values[e] in text_dict[key][i]:
                    count_no = count_emo(text_dict[key][i],emo_values[e])
                    list_emo.append([count_no,meaning_values[e]])
                    del_emo_list[i]=del_emo_list[i].replace(emo_values[e], " ")
            
            list_all_emo.append(list_emo)

        calculate_emo[key]=list_all_emo
        del_emo_dict[key]=del_emo_list
    
    return del_emo_dict,calculate_emo


In [16]:
# save the storage of emoji and emoticons
def save_data(name,data):
    pickle_file = open('DataSet\\'+name,mode='wb')
    pickle.dump(data,pickle_file)
    pickle_file.close()

In [17]:
text_3_0, calculate_emo_3 = extract_emo(text_3,emo_values,meaning_values)
text_all_0, calculate_emo_all = extract_emo(text_all,emo_values,meaning_values)

100%|██████████████████████████████████████████████████████████████████████████| 30160/30160 [00:06<00:00, 4317.63it/s]


In [18]:
calculate_emo_3

{'turn1': [[],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [[2, ' eye face love smile smiling face heart eyes ']],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [[1, ' smiley happy face ']],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [[1, ' tongue sticking cheeky playful blowing raspberry ']],
  [],
  [[1, ' unamused face '], [1, ' cold downcast face sweat face sweat ']],
  [],
  [[2, ' face face tears joy joy laugh tear ']],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [[1, ' break broken broken heart ']],
  [],
  [],
  [],
  [],
  [],
  [],
  [[1, ' face face steam nose triumph won ']],
  [],
  [],
  [],
  [],
  [[1, ' face grinning face big eyes mouth open smile ']],
  [],
  [[1, ' cat cat wry sm

In [19]:
save_path = "extraction_emo_3_{a}_{b}.pkl".format(a=path_change,b=version_change)
save_data(save_path,calculate_emo_3)

In [20]:
text_3_0

{'turn1': ["Don't worry  I'm girl",
  'When did I?',
  'By',
  'U r ridiculous',
  'Just for time pass',
  "I'm a dog person",
  'So whatsup',
  'Ok',
  'Really?',
  'Bay',
  'I hate my boyfriend',
  'I will do night.',
  'Sure go ahead',
  'Bad',
  'Ok get it......',
  'Money money and lots of money  ',
  'My gf left ne',
  'get lost',
  'You are lying and i know that',
  'Ur creator is very bad',
  'ehat is hehe',
  'Do you dance?',
  'I hate it too',
  'Not always',
  'Bcoz u dont know wat is to miss someone',
  'Yeah',
  'I want to tell you something',
  'Yes I will send the email and you write all yours filling still I write all matters but your are clever and not any matter for yourself please send andwrite',
  'You are very funny',
  'wth',
  'so rude',
  'ok i will',
  'Ntng.....',
  'You are fool',
  'U dare me?',
  'Abt me',
  'crazy',
  'Goa in India',
  'That was mean',
  'yes  ',
  "I don't no",
  'Send me your nekde pic',
  "OK friend's",
  'She is ignoring me',
  'Happy'

# 2.2. Lower the words

In [21]:
# lower the words
def lower_text(input_text):
    #lower the words
    input_text =input_text.lower()
    return input_text

In [22]:
text_3_1 = fit_transform(text_3_0,"lower_text")
text_all_1 = fit_transform(text_all_0,"lower_text")

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 155878.47it/s]


## 2.3. Seperate the adhered words with space and remove redundant emoji, emoticons
 - e.g. "you?" ? and you are adhered together, which needs to be split
 - e.g. "yess i am crazyyy😂😂" I also need to split emoji from the word "crazyyy" as "i am crazyyy 😂 😂 "
 - and some emoji, emoticons can't be found in the emo_df, therefore, I delete them since it's impossible to explain them in word2vec

In [23]:
# functions for extracting emoji, emoticons and punctuations
def extract_emoji(input_text):
    emoji_list = re.findall(r'(?:[^a-zA-Z\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸])',input_text)
    return emoji_list
    
def extract_emoticon(input_text):
    emoticon_list = re.findall(r'[\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸][\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸]+',input_text)
    return emoticon_list

def extract_punctuation(input_text):
    'here I use for loop to split the double punctuations into single ones'
    punctuation_list = re.findall(r'[^\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸](\!|\?|\.+)[^\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸]',input_text)
    result_list=list()
    for each in punctuation_list:
        list_split=[each]
        if each !="...":
            list_split=list(each)
        for punc in list_split:
            result_list.append(punc)
    return result_list

# functions for assigning emoji, emoticons and punctuations space from each other
def assign_space(input_text):
    #emoji
    input_text = re.sub(r'(?:[^ a-zA-Z\d\/\*\:\)\.\?\^\;?\-_\,\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸])',' ',input_text)
    #emoticon
    input_text = re.sub(r'[\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸][\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸]+',' ',input_text)
    #punctuation
    input_text = re.sub(r'[^\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸](\!|\?|\.+)[^\d\/\*\:\)\.\?\^\;?\-_\'~!\<\>\=\"#&$%\\\{\}\|\[\]ç\+ω○\@¡éı・…¡\`：）♡ӳ！“”à≧∇≦♂ş≈¬⊄─✔•×ü–₹。ó°ʖ—¶ķñ฿ĺ∑；⏸]',' \g<0> ' ,input_text)
    #replace the redundant space
    input_text = re.sub(r'[a-zA-Z]+',r' \g<0> ',input_text)
    input_text = re.sub(r'  +',r' ',input_text)
    return input_text

In [24]:
text_3_2 = fit_transform(text_3_1,"assign_space")
text_all_2 = fit_transform(text_all_1,"assign_space")

100%|█████████████████████████████████████████████████████████████████████████| 30160/30160 [00:01<00:00, 25670.59it/s]


# 3. Preprocessing: Correct the words in sentence
## 3.1. Correct common words

In [25]:
# turn don't, doesn't, can't, couldn't, shouldn't, didn't, isn't, wasn't, weren't, aren't, won't, shan't, 's, 're
# as do not, do not, can not, could not, should not, did not, is not, was not, were not, are not, will not, shall not, 's, are

# turn r , u, bt, n, 
# as are, you ,but, and
cor_dict= {
    "don ' t":      "do not",
    "doesn ' t":    "do not",
    "can ' t":      "can not",
    "couldn ' t":   "could not",
    "shouldn ' t":  "should not",
    "didn ' t":     "did not",
    "isn ' t":      "is not",
    "wasn ' t":     "was not",
    "weren ' t":    "were not",
    "aren ' t":     "are not",
    "won ' t":      "will not",
    "shan ' t":     "shall not",
    
    #"\' re ":       "are ",
    #" re ":         "are",
    "\' m ":        "am ",
    "\' ll ":       "will ",
    #" \' s ":       " \'s "

    }

In [26]:
def correct_words_common(input_text):
    'for correcting the words in sentences based on hand-crafted data'
    corrected_sentence = input_text
    for each_key in cor_dict.keys():
        if each_key in input_text:
            corrected_sentence = corrected_sentence.replace(each_key,cor_dict[each_key])
    return corrected_sentence


In [27]:
text_3_3 = fit_transform(text_3_2,"correct_words_common")
text_all_3 = fit_transform(text_all_2,"correct_words_common")

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 110771.73it/s]


## 3.2. Correct missing characters
- e.g. "hmmm i can talk no w", here "now" is missing "w".

In [28]:
def correct_words_combin(input_text):
    corrected_sentence = input_text
    corrected_sentence = re.sub(r' ([a-z]+) ([bcdefghjklnopqrstvwxz]) ',r' \g<1>\g<2> ',corrected_sentence)
    return corrected_sentence

In [29]:
testset = {"turn1":["I ' m go d , and I wasn ' t player wai t ", "hhh you ' re kidding m e "]}
print(fit_transform(testset,"correct_words_combin"))

100%|████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]

{'turn1': ["I ' m god , and I wasn ' t player wait ", "hhh you ' re kidding me "]}


In [30]:
text_3_4 = fit_transform(text_3_3,"correct_words_combin")
text_all_4 = fit_transform(text_all_3,"correct_words_combin")

100%|█████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 86159.83it/s]


In [31]:
text_all_4["all"][:10]

[" do not worry i am girl hmm how do i know if you are what ' s ur name ?",
 ' when did i ? saw many times i thinkn o . i never saw you ',
 ' by by google chrome where you live ',
 ' ur ridiculous i might be ridiculous but i am telling the truth . u little disgusting whore ',
 ' just for time pass wt do u do 4 a living then maybe ',
 ' i am a dog person youre so rude whaaaat why ',
 ' so whatsup nothing much . sitting sipping and watchingt v . how abt u ? what are you watching on tv ?',
 ' ok ok im back so , how are u ',
 ' reall y ? really really really really really y saying so many times i can hear you ',
 ' bay in the bay love you ']

## 3.3. Remove punctuations

In [32]:
def remove_punc(input_text):
    new_sentence = input_text
    new_sentence = re.sub(r'[^ a-zA-Z]',"",new_sentence).strip()
    return new_sentence

In [33]:
text_3_5 = fit_transform(text_3_4,"remove_punc")
text_all_5 = fit_transform(text_all_4,"remove_punc")

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 115422.64it/s]


In [34]:
text_all_5["all"][:10]

['do not worry i am girl hmm how do i know if you are what  s ur name',
 'when did i  saw many times i thinkn o  i never saw you',
 'by by google chrome where you live',
 'ur ridiculous i might be ridiculous but i am telling the truth  u little disgusting whore',
 'just for time pass wt do u do  a living then maybe',
 'i am a dog person youre so rude whaaaat why',
 'so whatsup nothing much  sitting sipping and watchingt v  how abt u  what are you watching on tv',
 'ok ok im back so  how are u',
 'reall y  really really really really really y saying so many times i can hear you',
 'bay in the bay love you']

## 3.4.  Remove stopwords made handcrafted

In [35]:
print(stopwords.words('english')) # which is not suitable here, coz e.g. in emotion analysis, not happy is opposite from happy.

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [36]:
stopwords_byhand = ["to","a","and","of"]

In [37]:
def remove_stopwords(input_text):

    words = input_text.split()
    new_words=list()
    for word in words:
        if word in stopwords_byhand:
            continue
        else:
            new_words.append(word)
    sentence = " ".join(new_words)
  
    return sentence

In [38]:
text_3_6 = fit_transform(text_3_5,"remove_stopwords")
text_all_6 = fit_transform(text_all_5,"remove_stopwords")

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 101478.54it/s]


# 4. Preprocessing: Correction of Incorrect Spell and Shorthand Words
## 4.1. Correcting spell mistake
 - e.g. "nt" is actually "not", "noo" is actually "no"
 - in order to do so, here i use library called pyspellchecker

In [39]:
from spellchecker import SpellChecker
spell = SpellChecker()
def correct_spell(input_text):

    words = input_text.split()
    new_words=list()
    for word in words:
        word = spell.correction(word)
        new_words.append(word)
    sentence = " ".join(new_words)
    return sentence


In [40]:
text_3_7 = fit_transform(text_3_6,"correct_spell")
text_all_7 = fit_transform(text_all_6,"correct_spell")

100%|████████████████████████████████████████████████████████████████████████████| 30160/30160 [22:29<00:00, 22.35it/s]


In [41]:
text_all_7 ["all"][:10]

['do not worry i am girl hmm how do i know if you are what s ur name',
 'when did i saw many times i think o i never saw you',
 'by by goole chrome where you live',
 'ur ridiculous i might be ridiculous but i am telling the truth u little disgusting whore',
 'just for time pass wt do u do living then maybe',
 'i am dog person your so rude whaaaat why',
 'so whatsit nothing much sitting sipping watching v how at u what are you watching on tv',
 'ok ok im back so how are u',
 'really y really really really really really y saying so many times i can hear you',
 'bay in the bay love you']

In [42]:
# after spell mistakce correction, it's needed to run over again to remove the stopwords
text_3_8 = fit_transform(text_3_7,"remove_stopwords")
text_all_8 = fit_transform(text_all_7,"remove_stopwords")

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 107627.14it/s]


## 4.2. shorthand words translation
- here we only deal with the most frequent words happened in text which can't be changed into vectors based on pretrained word2vec index

In [43]:
cor_dict2= {
    "goole" : "google",
    "favourite":"favorite",
    "he/she":"he or she",
    "humour":"humor",
    "ur":"your",
    "u":"you",
    "s":"is",
    "budd":"buddy",
    "colour":"color",
    "emosi":"emoji",
    "programme":"program",
    "travelling":"travel",
    "aaaaa":"ah",
    "shut-up":"shut up",
    "favourites":"favorite",
    "oho":"oh",
    "learnt":"learn",
    "wasnt":"was not",
    "ha-ha":"ha ha",
    "naaah":"no",
    "moodie":"moody",
    "ah-ha":"ah ha",
    "wowww":"wow",
    "okkkkk":"ok",
    "hahahha":"ha ha ha",
    "frida":"friday",
    "friendzone":"friend zone",
    "friendzoned":"friend zone",
    "realise":"realize",
    "computerised":"computerise",
    "apologise":"apologize",
    "youuuu":"you",
    "starbuck":"starbucks",
    "realised":"realize",
    "judgement":"judge",
    "honoured":"honor",
    "organisation":"organization",
    "ddl":"deadline",
    "wowwww":"wow",
    }

In [44]:
def correct_words_common2(input_text):
    'for correcting the words in sentences based on hand-crafted data'
    corrected_sentence = " "+input_text+" "
    for each_key in cor_dict2.keys():
        if each_key in input_text:
            rule = " "+each_key+" "
            corrected_sentence = re.sub(rule," "+cor_dict2[each_key]+" ",corrected_sentence).strip()
            #corrected_sentence.replace(each_key,cor_dict2[each_key])
    return corrected_sentence

In [45]:
text_3_9 = fit_transform(text_3_8,"correct_words_common2")
text_all_9 = fit_transform(text_all_8,"correct_words_common2")

100%|█████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 70815.66it/s]


# 5. check data format 
- for further preprocessing based on pre-trained embedding resource

## 5.1. check distinct vocabulary

In [46]:
def distinct_words(dialogs):
    vocab = dict()
    for each_dialog in tqdm(dialogs):
        words_list = each_dialog.split(sep=" ")
        for word in words_list:
            if word == '':
                continue
            try:
                vocab[word] +=1
            except KeyError:
                vocab[word] = 1
    
    return vocab

In [47]:
vocab = distinct_words(text_all["all"])
print("the length of distinct vocabulary of {a} is {b}".format(a="the version original",b=len(vocab)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 252009.52it/s]

the length of distinct vocabulary of the version original is 33024


In [48]:
vocab0 = distinct_words(text_all_0["all"])
print("the length of distinct vocabulary of {a} is {b}".format(a="the version 0",b=len(vocab0)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 254129.26it/s]

the length of distinct vocabulary of the version 0 is 30589


In [49]:
vocab1 = distinct_words(text_all_1["all"])
print("the length of distinct vocabulary of {a} is {b}".format(a="the version 1",b=len(vocab1)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 258466.47it/s]

the length of distinct vocabulary of the version 1 is 25453


In [50]:
vocab2 = distinct_words(text_all_2["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 2",b=len(vocab2)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 238114.90it/s]

the number of distinct vocabulary of the version 2 of data is 15601


In [51]:
vocab3 = distinct_words(text_all_3["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 3",b=len(vocab3)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 247873.40it/s]

the number of distinct vocabulary of the version 3 of data is 15597


In [52]:
vocab4 = distinct_words(text_all_4["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 4",b=len(vocab3)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 238117.14it/s]

the number of distinct vocabulary of the version 4 of data is 15597


In [53]:
vocab5 = distinct_words(text_all_5["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 5",b=len(vocab5)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 274814.65it/s]

the number of distinct vocabulary of the version 5 of data is 15307


In [54]:
vocab6 = distinct_words(text_all_6["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 6",b=len(vocab6)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 296374.19it/s]

the number of distinct vocabulary of the version 6 of data is 15303


In [55]:
vocab7 = distinct_words(text_all_7["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 7",b=len(vocab7)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 288005.79it/s]

the number of distinct vocabulary of the version 7 of data is 11272


In [56]:
vocab8 = distinct_words(text_all_8["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 8",b=len(vocab8)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 293595.93it/s]

the number of distinct vocabulary of the version 8 of data is 11268


In [57]:
vocab9 = distinct_words(text_all_9["all"])
print("the number of distinct vocabulary of {a} of data is {b}".format(a="the version 9",b=len(vocab9)))

100%|████████████████████████████████████████████████████████████████████████| 30160/30160 [00:00<00:00, 290690.81it/s]

the number of distinct vocabulary of the version 9 of data is 11258


## 5.2. check the percentage of words in data can be processed by word2vec pre-trained data

In [58]:
# import embedding resource
pretrained_word2vec_path = "DataSet\\GoogleNews-vectors-negative300.bin"
word2vec_pre = gs.models.KeyedVectors.load_word2vec_format(pretrained_word2vec_path, binary= True)

In [59]:
# build the function for check the coverage of the vocabulary and the text
import operator
def check_coverage(vocab, embedding_resource):
    cov_vocab =0
    num_vocab = len(vocab)
    cov_text = 0
    not_cov = dict()
    not_text = 0
    for word in tqdm(vocab):
        try:
            x = embedding_resource[word]
            cov_vocab += 1
            cov_text += vocab[word]
        except:
            not_cov[word]=vocab[word]
            not_text += vocab[word]
            pass
    percent_cov_vocab = cov_vocab/num_vocab
    percent_cov_text = cov_text/(cov_text+not_text)
    print("In Embedding Index we have {:.2%} coverage of distinct vocabulary".format(percent_cov_vocab))
    print("And we have {:.2%} coverage of all text".format(percent_cov_text))
    sorted_not_cov = sorted(not_cov.items(),key= operator.itemgetter(1),reverse = True)
    print("The number of words which are not covered in word2vec resource is: {0}".format(len(sorted_not_cov)))
    return sorted_not_cov

In [60]:
# original version of data
not_cov = check_coverage(vocab, word2vec_pre)

100%|████████████████████████████████████████████████████████████████████████| 33024/33024 [00:00<00:00, 533728.53it/s]

In Embedding Index we have 41.52% coverage of distinct vocabulary
And we have 82.72% coverage of all text
The number of words which are not covered in word2vec resource is: 19311


In [61]:
not_cov[:20]

[('to', 7659),
 ('a', 5278),
 ('and', 2486),
 ('of', 2444),
 ('?', 1286),
 ('you?', 651),
 ('😂', 601),
 (':(', 512),
 ('what?', 465),
 ('.', 380),
 ('you.', 374),
 (':D', 348),
 (':P', 346),
 ('!', 277),
 ('😂😂', 272),
 ('me.', 254),
 ('me?', 246),
 ('??', 239),
 ('why?', 222),
 ('it?', 202)]

In [62]:
# remove emoji, emoticon
not_cov0 = check_coverage(vocab0, word2vec_pre)

100%|████████████████████████████████████████████████████████████████████████| 30589/30589 [00:00<00:00, 479251.89it/s]

In Embedding Index we have 44.89% coverage of distinct vocabulary
And we have 84.51% coverage of all text
The number of words which are not covered in word2vec resource is: 16857


In [63]:
not_cov0[:20]
# here we can see that the trained word2vec resource can't cover some stopwords, punctuations and the combination with word and punctuation

[('to', 7665),
 ('a', 5278),
 ('and', 2489),
 ('of', 2445),
 ('?', 1301),
 ('you?', 654),
 ('what?', 468),
 ('.', 404),
 ('you.', 378),
 ('!', 283),
 ('me.', 257),
 ('me?', 249),
 ('??', 240),
 ('why?', 223),
 ('it?', 202),
 ('Why?', 194),
 ('...', 184),
 ('you!', 176),
 (',', 173),
 ('..', 161)]

In [64]:
# lower the words
not_cov1 = check_coverage(vocab1, word2vec_pre)

100%|████████████████████████████████████████████████████████████████████████| 25453/25453 [00:00<00:00, 519926.85it/s]

In Embedding Index we have 38.46% coverage of distinct vocabulary
And we have 83.43% coverage of all text
The number of words which are not covered in word2vec resource is: 15664


In [65]:
not_cov1[:20]
# here we can see that word2vec resource can't cover the words that are not correct spelled.

[('to', 7925),
 ('a', 5535),
 ('and', 3080),
 ("i'm", 2655),
 ('of', 2549),
 ('?', 1301),
 ('you?', 699),
 ('what?', 634),
 ('why?', 437),
 ('.', 404),
 ('you.', 398),
 ("i'll", 360),
 ('!', 283),
 ('me.', 266),
 ('me?', 260),
 ('??', 240),
 ('it?', 207),
 ('you!', 186),
 ('...', 184),
 (',', 173)]

In [66]:
# seperate adhered words such as "you!!!" with space into "you ! ! ! "
not_cov2 = check_coverage(vocab2, word2vec_pre)

100%|████████████████████████████████████████████████████████████████████████| 15601/15601 [00:00<00:00, 521413.72it/s]

In Embedding Index we have 69.75% coverage of distinct vocabulary
And we have 84.77% coverage of all text
The number of words which are not covered in word2vec resource is: 4719


In [67]:
not_cov2[:20] #now we can see the adhered words pair such as "you!!!" are gone

[("'", 13180),
 ('.', 11437),
 ('?', 10463),
 ('to', 8131),
 ('a', 6020),
 (',', 3967),
 ('!', 3484),
 ('and', 3118),
 ('of', 2574),
 ('"', 346),
 ('-', 266),
 (';', 213),
 (':', 167),
 (')', 131),
 ('/', 103),
 ('funn', 85),
 ('happene', 83),
 ('didnt', 77),
 ('sorr', 74),
 ('favourite', 73)]

In [68]:
# correct common words e.g. "you're" as "you are"
not_cov3 = check_coverage(vocab3, word2vec_pre)

100%|████████████████████████████████████████████████████████████████████████| 15597/15597 [00:00<00:00, 458257.57it/s]

In Embedding Index we have 69.75% coverage of distinct vocabulary
And we have 86.20% coverage of all text
The number of words which are not covered in word2vec resource is: 4718


In [69]:
not_cov3[:20]

[('.', 11437),
 ('?', 10463),
 ('to', 8131),
 ('a', 6020),
 ("'", 5316),
 (',', 3967),
 ('!', 3484),
 ('and', 3118),
 ('of', 2574),
 ('"', 346),
 ('-', 266),
 (';', 213),
 (':', 167),
 (')', 131),
 ('/', 103),
 ('funn', 85),
 ('happene', 83),
 ('didnt', 77),
 ('sorr', 74),
 ('favourite', 73)]

In [70]:
# add missing charactors e.g. "hmmm i can talk no w", here "now" is missing "w".
not_cov4 = check_coverage(vocab4, word2vec_pre)

100%|████████████████████████████████████████████████████████████████████████| 15400/15400 [00:00<00:00, 466744.33it/s]

In Embedding Index we have 69.49% coverage of distinct vocabulary
And we have 85.85% coverage of all text
The number of words which are not covered in word2vec resource is: 4698


In [71]:
not_cov4[2000:2020]

[('andl', 1),
 ('alomr', 1),
 ('velapan', 1),
 ('talket', 1),
 ('hegels', 1),
 ('veryf', 1),
 ('themk', 1),
 ('differenceb', 1),
 ('tyrion', 1),
 ('vxxvgfeecbko', 1),
 ('geniusk', 1),
 ('ttyt', 1),
 ('variet', 1),
 ('satisfactor', 1),
 ('vcook', 1),
 ('sleepyp', 1),
 ('noonnono', 1),
 ('tryyy', 1),
 ('plesure', 1),
 ('februar', 1)]

In [72]:
# remove punctuation
not_cov5 = check_coverage(vocab5, word2vec_pre)

100%|████████████████████████████████████████████████████████████████████████| 15307/15307 [00:00<00:00, 510964.76it/s]

In Embedding Index we have 69.69% coverage of distinct vocabulary
And we have 93.37% coverage of all text
The number of words which are not covered in word2vec resource is: 4639


In [73]:
not_cov5[:20]

[('to', 7875),
 ('a', 5975),
 ('and', 3115),
 ('of', 2548),
 ('funn', 85),
 ('didnt', 82),
 ('favourite', 79),
 ('sorr', 74),
 ('howr', 56),
 ('ohk', 50),
 ('whatsapp', 49),
 ('yess', 49),
 ('whatr', 42),
 ('angr', 37),
 ('doesnt', 34),
 ('whyr', 33),
 ('okk', 31),
 ('wherer', 31),
 ('plzz', 30),
 ('bcz', 30)]

In [74]:
# Remove stopwords made handcrafted
not_cov6 = check_coverage(vocab6, word2vec_pre)

100%|████████████████████████████████████████████████████████████████████████| 15303/15303 [00:00<00:00, 528623.24it/s]

In Embedding Index we have 69.71% coverage of distinct vocabulary
And we have 98.05% coverage of all text
The number of words which are not covered in word2vec resource is: 4635


In [75]:
not_cov6[:20]

[('funn', 85),
 ('didnt', 82),
 ('favourite', 79),
 ('sorr', 74),
 ('howr', 56),
 ('ohk', 50),
 ('whatsapp', 49),
 ('yess', 49),
 ('whatr', 42),
 ('angr', 37),
 ('doesnt', 34),
 ('whyr', 33),
 ('okk', 31),
 ('wherer', 31),
 ('plzz', 30),
 ('bcz', 30),
 ('erience', 29),
 ('humour', 28),
 ('colour', 26),
 ('seriousl', 23)]

In [76]:
# correct misspell
not_cov7 = check_coverage(vocab7, word2vec_pre)

100%|████████████████████████████████████████████████████████████████████████| 11272/11272 [00:00<00:00, 491390.92it/s]

In Embedding Index we have 88.78% coverage of distinct vocabulary
And we have 99.31% coverage of all text
The number of words which are not covered in word2vec resource is: 1265


In [77]:
not_cov7[:20]

[('of', 244),
 ('goole', 90),
 ('favourite', 81),
 ('to', 61),
 ('whatsapp', 49),
 ('thnn', 48),
 ('boz', 41),
 ('emosi', 40),
 ('a', 40),
 ('dumas', 33),
 ('he/she', 31),
 ('and', 30),
 ('humour', 29),
 ('colour', 27),
 ('budd', 20),
 ('ranbir', 19),
 ('programme', 16),
 ('travelling', 13),
 ('wasnt', 12),
 ('aaaaa', 11)]

In [78]:
# remove stopwords again after correction of missepll
not_cov8 = check_coverage(vocab8, word2vec_pre)

100%|████████████████████████████████████████████████████████████████████████| 11268/11268 [00:00<00:00, 470787.52it/s]

In Embedding Index we have 88.81% coverage of distinct vocabulary
And we have 99.40% coverage of all text
The number of words which are not covered in word2vec resource is: 1261


In [79]:
not_cov8[:20]

[('goole', 90),
 ('favourite', 81),
 ('whatsapp', 49),
 ('thnn', 48),
 ('boz', 41),
 ('emosi', 40),
 ('dumas', 33),
 ('he/she', 31),
 ('humour', 29),
 ('colour', 27),
 ('budd', 20),
 ('ranbir', 19),
 ('programme', 16),
 ('travelling', 13),
 ('wasnt', 12),
 ('aaaaa', 11),
 ('shut-up', 11),
 ('trw', 11),
 ('oho', 10),
 ('fyn', 10)]

In [80]:
# correct shorthand words or some misspell which can't be corrected by pyspellchecker
not_cov9 = check_coverage(vocab9, word2vec_pre)

100%|████████████████████████████████████████████████████████████████████████| 11258/11258 [00:00<00:00, 490780.61it/s]

In Embedding Index we have 88.91% coverage of distinct vocabulary
And we have 99.52% coverage of all text
The number of words which are not covered in word2vec resource is: 1249


In [81]:
not_cov9[:40]

[('whatsapp', 49),
 ('thnn', 48),
 ('boz', 41),
 ('dumas', 33),
 ('ranbir', 19),
 ('trw', 11),
 ('fyn', 10),
 ('colour', 8),
 ('bessie', 8),
 ('cortina', 8),
 ('helena', 8),
 ('pyy', 8),
 ('natasha', 8),
 ('aaaaa', 7),
 ('emosi', 7),
 ('grey', 7),
 ('bahubali', 7),
 ('sheehan', 7),
 ('tajmahal', 7),
 ('alida', 6),
 ('sinfield', 6),
 ('malaya', 6),
 ('krk', 5),
 ('wowwww', 5),
 ('programme', 5),
 ('planation', 5),
 ('jarvis', 5),
 ('favourites', 5),
 ('gujarati', 5),
 ('raoul', 5),
 ('travelling', 5),
 ('usl', 5),
 ('samoa', 5),
 ('wcm', 4),
 ('deepika', 4),
 ('mohanlal', 4),
 ('whatsaap', 4),
 ('karol', 4),
 ('didcot', 4),
 ('mrna', 4)]

# 6. data storage

In [82]:
# finally we have 2 versions of preprocessing of data so far
# first one is preprocessed train data without spelling and shorthand words correction, 
# but after assigning space, correction of common shorthand words, adding missing charactors, removal of stopwords, emoji, emoticon and punctuations
# the second one is preprocessed train data with spelling and shorthand words correction.
# and I also have extracted all the emoji and emoticons of the data, which can be later processed for further analysis

In [83]:
def save_data(name,data):
    pickle_file = open('DataSet\\'+name,mode='wb')
    pickle.dump(data,pickle_file)
    pickle_file.close()

In [84]:
#save_data("pre_train_3turn_no_correct_word2vec.pkl",text_3_6)
save_path3_6 = "pre_{a}_3turn_no_correct_{b}.pkl".format(a=path_change,b=version_change)
save_patha_6 = "pre_{a}_all_no_correct_{b}.pkl".format(a=path_change,b=version_change)
save_path3_9 = "pre_{a}_3turn_no_emo_{b}.pkl".format(a=path_change,b=version_change)
save_patha_9 = "pre_{a}_all_no_emo_{b}.pkl".format(a=path_change,b=version_change)
save_data(save_path3_6,text_3_6)
save_data(save_patha_6 ,text_all_6)
save_data(save_path3_9,text_3_9)
save_data(save_patha_9,text_all_9)